In [1]:
import numpy as np
import pandas as pd

import utils
import default_parameters
import make_charts
import meet_indicators
import ca_reopening_tiers

from IPython.display import display_html, Markdown 

# Default parameters
county_state_name = default_parameters.county_state_name
state_name = default_parameters.state_name
msa_name = default_parameters.msa_name
time_zone = default_parameters.time_zone

fulldate_format = default_parameters.fulldate_format
monthdate_format = default_parameters.monthdate_format
start_date = default_parameters.start_date
yesterday_date = default_parameters.yesterday_date
today_date = default_parameters.today_date
one_week_ago = default_parameters.one_week_ago
two_weeks_ago = default_parameters.two_weeks_ago
three_weeks_ago = default_parameters.three_weeks_ago
two_days_ago = default_parameters.two_days_ago
eight_days_ago = default_parameters.eight_days_ago

# Set cut-offs
county_test_lower_bound = 15_000
county_test_upper_bound = 16_667
positive_lower_bound = 0.04
positive_upper_bound = 0.08
positive_2weeks_bound = 0.05
hospital_bound = 0.30
ca_hospitalization_bound = 0.05

# Set cut-offs for CA reopening
ca_case_minimal_bound = 1
ca_case_moderate_bound = 4
ca_case_substantial_bound = 7 

ca_test_minimal_bound = 0.020
ca_test_moderate_bound = 0.050
ca_test_substantial_bound = 0.080

In [2]:
a1 = ca_reopening_tiers.case_rate(county_state_name, start_date, "today")
a2 = ca_reopening_tiers.case_rate(county_state_name, start_date, "one_week_ago")
a3 = ca_reopening_tiers.case_rate(county_state_name, start_date, "two_weeks_ago")

print(a1)
print(a2)
print(a3)

8.13
9.82
12.36


In [3]:
b1 = ca_reopening_tiers.positive_rate(start_date, "one_week_ago")
b2 = ca_reopening_tiers.positive_rate(start_date, "two_weeks_ago")

print(b1)
print(b2)

0.034
0.039


In [4]:
ca_county = pd.DataFrame(
    {"case": [a1, a2, a3], 
     "test": [np.nan, b1, b2]
    }
)

ca_county

,case,test
0,8.13,NaN
1,9.82,0.034
2,12.36,0.039


In [5]:
def put_in_tiers(row, indicator, minimal_bound, moderate_bound, substantial_bound):
    if row[indicator] < minimal_bound:
        return 1
    elif (row[indicator] >= minimal_bound) and (row[indicator] < moderate_bound):
        return 2
    elif (row[indicator] >= moderate_bound) and (row[indicator] <= substantial_bound):
        return 3
    elif row[indicator] > substantial_bound:
        return 4
    else: 
        return np.nan

def add_tiers(df):
    df = df.assign(
        case_tier = df.apply(lambda x: 
                                    put_in_tiers(x, "case", 
                                    ca_case_minimal_bound, ca_case_moderate_bound, 
                                    ca_case_substantial_bound), axis=1).astype("Int64"), 
        test_tier = df.apply(lambda x: 
                                    put_in_tiers(x, "test", 
                                    ca_test_minimal_bound, ca_test_moderate_bound, 
                                    ca_test_substantial_bound), axis=1).astype("Int64")
    )
    
    # If 2 indicators belong in different tiers, most restrictive (max) is assigned overall
    df = df.assign(
            overall_tier = df[["case_tier", "test_tier"]].max(axis=1).astype(int)
    )[["overall_tier", "case", "test"]]
    
    # Transpose
    df = (df.T
          .rename(columns = {0: "current_week", 
                             1: "last_week", 
                             2: "two_weeks_ago",})
    )
    
    return df

In [6]:
ca_county = add_tiers(ca_county)
ca_county

,current_week,last_week,two_weeks_ago
overall_tier,4.00,4.000,4.000
case,8.13,9.820,12.360
test,NaN,0.034,0.039


In [7]:
# Add CA Blueprint for Safer Economy (reopening, take 2) criterion to get colors
def check_ca_case(row):
    status = ["minimal" if x < ca_case_minimal_bound 
              else "moderate" if ((x >= ca_case_minimal_bound) and (x < ca_case_moderate_bound))
              else "substantial" if ((x >= ca_case_moderate_bound) and (x <= ca_case_substantial_bound))
              else "widespread" if x > ca_case_substantial_bound 
              else "" for x in row] 
    
    return pd.Series(status, index=row.index, dtype="category") 

def check_ca_test(row):
    status = ["minimal" if x < ca_test_minimal_bound 
              else "moderate" if (x >= ca_test_minimal_bound) and (x < ca_test_moderate_bound)
              else "substantial" if (x >= ca_test_moderate_bound) and (x <= ca_test_substantial_bound)
              else "widespread" if x > ca_test_substantial_bound 
              else "" for x in row] 
    
    return pd.Series(status, index=row.index, dtype="category") 

def check_ca_overall_tier(row):
    status = ["minimal" if x==1 
              else "moderate" if x==2
              else "substantial" if x==3
              else "widespread" if x==4
              else "" for x in row] 
    
    return pd.Series(status, index=row.index, dtype="category") 

In [8]:
ca_tier_fnmap = {
    "overall_tier": check_ca_overall_tier,
    "case": check_ca_case,
    "test": check_ca_test,
}

purple = make_charts.purple
red = make_charts.maroon
orange = make_charts.orange
yellow = make_charts.yellow

stylemap = {
    "widespread": f"background-color: {purple}; color: white; font-weight: bold; opacity: 0.7" ,
    "substantial": f"background-color: {red}; color: white; font-weight: bold; opacity: 0.7",
    "moderate": f"background-color: {orange}; color: white; font-weight: bold; opacity: 0.7",
    "minimal": f"background-color: {yellow}; color: white; font-weight: bold; opacity: 0.7",    
    "": "background-color: white; color: black; font-weight: bold; opacity: 0.7",
}

In [9]:
(ca_county.style
   .format(lambda s: f"{s:,g}", na_rep="-")
   .apply(lambda row: ca_tier_fnmap[row.name](row).map(stylemap), axis=1)
   #.render()
  )

,current_week,last_week,two_weeks_ago
overall_tier,4,4,4
case,8.13,9.82,12.36
test,-,0.034,0.039
